In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import os
import pandas as pd
import datetime
import requests
from bs4 import BeautifulSoup as soup

In [2]:
os.chdir("### INPUT YOUR OWN PATH ###')

In [6]:
### importing a chrome driver
driver = webdriver.Chrome(r'### INPUT YOUR OWN PATH ###\chromedriver_win32\chromedriver.exe')
### go to site
driver.get('https://ktb.moef.go.kr/eng/aucRes.do')

### we are scraping for only 10 pages, you can manually adjust it to the number of pages that you need
for page in range(0,10):
    for tab in range(4,4+4):
        for number in range(1,11):
            try:
                title = driver.find_element_by_xpath('//*[@id="frm"]/table/tbody/tr['+str(number)+']/td[2]/a').text
                ### we only take the info on the regular/ normal auctions
                if title[-4:].isnumeric() == True: 
                    driver.find_element_by_xpath('//*[@id="frm"]/table/tbody/tr['+str(number)+']/td[2]/a').click()
                    ### this finds the number of rows in the table
                    elems = driver.find_elements_by_xpath("//table/tbody/tr")
                    for i in elems[1].text.split('\n'):
                        if 'Auction date' in i:
                            start_row = elems[1].text.split('\n').index(i) 
                        if '% of the amount of accepted competitive tenders' in i:
                            end_row = (elems[1].text.split('\n').index(i))
                    num_of_rows = (end_row-start_row)/2+1
                    item = []
                    value = []
                    for row in range(1,int(num_of_rows)+1):
                        item.append(driver.find_element_by_xpath('//*[@id="frm"]/div/table/tbody/tr[2]/td/div/table/tbody/tr['+str(row)+']/td[1]').text)
                        value.append(driver.find_element_by_xpath('//*[@id="frm"]/div/table/tbody/tr[2]/td/div/table/tbody/tr['+str(row)+']/td[2]').text)
                    df = pd.DataFrame(value, columns=['Value'], index=item)
                    date = datetime.datetime.strptime(value[1],"%A, %B %d, %Y")
                    final_date = str(date.year)+'-'+str(date.month)+'-'+str(date.day)     
                    df.to_csv(final_date+'.csv')
                    time.sleep(1)
                    driver.back()
            ### if for whatever reason the code doesn't work for that page, just skip it and proceed to the next page
            except:
                driver.back()
        ### this clicks the next tab
        driver.find_element_by_xpath('//*[@id="frm"]/ol/li['+str(tab)+']/a').click()
    ### this clicks the >
    driver.find_element_by_xpath('//*[@id="frm"]/ol/li[8]/a').click()

In [10]:
import glob
import dateutil

os.chdir(r'### INPUT YOUR OWN PATH ###')
df = pd.read_csv(glob.glob('*.csv')[0],index_col=0)

for name in glob.glob('*.csv')[1:]:
    df2 = pd.read_csv(name,index_col=0)
    df = pd.merge(df, df2, left_index=True, right_index=True)

df.columns = [dateutil.parser.parse(i[:-4]) for i in glob.glob('*.csv')]
df = df.sort_index(axis=1)
display(df)
df.to_csv('Aggregate.csv')

,2019-01-21,2019-05-14,2019-05-21,2019-05-28,2019-06-04,2019-06-07,2019-06-11,2019-06-18,2019-06-25,2019-07-02,...,2021-12-09,2021-12-13,2021-12-14,2021-12-21,2021-12-22,2022-01-04,2022-01-11,2022-01-12,2022-01-17,2022-01-18
Auction date (time),"Monday, January 20, 2020 (10:40~11:00)","Monday, May 13, 2019 (10:40~11:00)","Monday, May 20, 2019 (10:40~11:00)","Monday, May 27, 2019 (10:40~11:00)","Monday, June 3, 2019 (10:40~11:00)","Tuesday, June 4, 2019 (10:40~11:00)","Monday, June 10, 2019 (10:40~11:00)","Monday, June 17, 2019 (10:40~11:00)","Monday, June 24, 2019 (10:40~11:00)","Monday, July 1, 2019 (10:40~11:00)",...,"Tuesday, December 7, 2021 (10:40~11:00)","Friday, December 10, 2021 (10:40~11:00)","Monday, December 13, 2021 (10:40~11:00)","Monday, December 20, 2021 (10:40~11:00)","Tuesday, December 21, 2021 (10:40~11:00)","Monday, January 3, 2022 (10:40~11:00)","Monday, January 10, 2022 (10:40~11:00)","Tuesday, January 11, 2022 (10:40~11:00)","Friday, January 14, 2022 (10:40~11:00)","Monday, January 17, 2022 (10:40~11:00)"
Settlement date,"Tuesday, January 21, 2019","Tuesday, May 14, 2019","Tuesday, May 21, 2019","Tuesday, May 28, 2019","Tuesday, June 4, 2019","Friday, June 7, 2019","Tuesday, June 11, 2019","Tuesday, June 18, 2019","Tuesday, June 25, 2019","Tuesday, July 2, 2019",...,"Thursday, December 9, 2021","Monday, December 13, 2021","Tuesday, December 14, 2021","Tuesday, December 21, 2021","Wednesday, December 22, 2021","Tuesday, January 4, 2022","Tuesday, January 11, 2022","Wednesday, January 12, 2022","Monday, January 17, 2022","Tuesday, January 18, 2022"
Issue date,"December 10, 2019","March 10, 2019","December 10, 2018","September 10, 2018","June 10, 2019","March 10, 2019","March 10, 2019","June 10, 2019","September 10, 2018","June 10, 2019",...,"December 10, 2021","September 10, 2021","December 10, 2021","September 10, 2021","September 10, 2021","March 10, 2021","December 10, 2021","December 10, 2021","September 10, 2021","December 10, 2021"
Coupon rate,1.375%,1.875%,2.375%,2.375%,1.625%,2.000%,1.875%,1.875%,2.375%,1.625%,...,1.750%,1.625%,2.375%,1.750%,1.875%,1.875%,1.875%,1.750%,1.625%,2.375%
Amount offered,"2,600 billion won","1,700 billion won","1,000 billion won",500 billion won,"1,650 billion won","1,850 billion won","1,650 billion won","2,150 billion won",500 billion won,"1,150 billion won",...,500 billion won,350 billion won,"1,000 billion won",950 billion won,300 billion won,"3,300 billion won","2,500 billion won",900 billion won,400 billion won,"2,600 billion won"
Amount received,"7,461 billion won","5,074 billion won","3,399 billion won","1,596 billion won","4,838 billion won","5,041 billion won","5,001 billion won","6,381 billion won","1,609 billion won","3,634 billion won",...,"1,411 billion won",558 billion won,"2,974 billion won","2,616 billion won",850 billion won,"8,576 billion won","6,945 billion won","2,660 billion won",847 billion won,"7,007 billion won"
Amount accepted,"2,602 billion won","1,700 billion won","1,000 billion won",508 billion won,"1,650 billion won","1,850 billion won","1,650 billion won","2,150 billion won",506 billion won,"1,650 billion won",...,500 billion won,350 billion won,"1,000 billion won",950 billion won,300 billion won,"3,318 billion won","2,549 billion won",904 billion won,437 billion won,"2,600 billion won"
Bid‐to‐Cover Ratio,287.0%,298.5%,339.9%,316.4%,293.2%,272.5%,303.1%,296.8%,321.8%,316.0%,...,282.2%,159.4%,297.4%,275.4%,283.3%,259.9%,277.8%,295.6%,211.8%,269.5%


,2019-01-21,2019-05-14,2019-05-21,2019-05-28,2019-06-04,2019-06-07,2019-06-11,2019-06-18,2019-06-25,2019-07-02,...,2021-12-09,2021-12-13,2021-12-14,2021-12-21,2021-12-22,2022-01-04,2022-01-11,2022-01-12,2022-01-17,2022-01-18
Auction date (time),"Monday, January 20, 2020 (10:40~11:00)","Monday, May 13, 2019 (10:40~11:00)","Monday, May 20, 2019 (10:40~11:00)","Monday, May 27, 2019 (10:40~11:00)","Monday, June 3, 2019 (10:40~11:00)","Tuesday, June 4, 2019 (10:40~11:00)","Monday, June 10, 2019 (10:40~11:00)","Monday, June 17, 2019 (10:40~11:00)","Monday, June 24, 2019 (10:40~11:00)","Monday, July 1, 2019 (10:40~11:00)",...,"Tuesday, December 7, 2021 (10:40~11:00)","Friday, December 10, 2021 (10:40~11:00)","Monday, December 13, 2021 (10:40~11:00)","Monday, December 20, 2021 (10:40~11:00)","Tuesday, December 21, 2021 (10:40~11:00)","Monday, January 3, 2022 (10:40~11:00)","Monday, January 10, 2022 (10:40~11:00)","Tuesday, January 11, 2022 (10:40~11:00)","Friday, January 14, 2022 (10:40~11:00)","Monday, January 17, 2022 (10:40~11:00)"
Settlement date,"Tuesday, January 21, 2019","Tuesday, May 14, 2019","Tuesday, May 21, 2019","Tuesday, May 28, 2019","Tuesday, June 4, 2019","Friday, June 7, 2019","Tuesday, June 11, 2019","Tuesday, June 18, 2019","Tuesday, June 25, 2019","Tuesday, July 2, 2019",...,"Thursday, December 9, 2021","Monday, December 13, 2021","Tuesday, December 14, 2021","Tuesday, December 21, 2021","Wednesday, December 22, 2021","Tuesday, January 4, 2022","Tuesday, January 11, 2022","Wednesday, January 12, 2022","Monday, January 17, 2022","Tuesday, January 18, 2022"
Issue date,"December 10, 2019","March 10, 2019","December 10, 2018","September 10, 2018","June 10, 2019","March 10, 2019","March 10, 2019","June 10, 2019","September 10, 2018","June 10, 2019",...,"December 10, 2021","September 10, 2021","December 10, 2021","September 10, 2021","September 10, 2021","March 10, 2021","December 10, 2021","December 10, 2021","September 10, 2021","December 10, 2021"
Coupon rate,1.375%,1.875%,2.375%,2.375%,1.625%,2.000%,1.875%,1.875%,2.375%,1.625%,...,1.750%,1.625%,2.375%,1.750%,1.875%,1.875%,1.875%,1.750%,1.625%,2.375%
Amount offered,"2,600 billion won","1,700 billion won","1,000 billion won",500 billion won,"1,650 billion won","1,850 billion won","1,650 billion won","2,150 billion won",500 billion won,"1,150 billion won",...,500 billion won,350 billion won,"1,000 billion won",950 billion won,300 billion won,"3,300 billion won","2,500 billion won",900 billion won,400 billion won,"2,600 billion won"
Amount received,"7,461 billion won","5,074 billion won","3,399 billion won","1,596 billion won","4,838 billion won","5,041 billion won","5,001 billion won","6,381 billion won","1,609 billion won","3,634 billion won",...,"1,411 billion won",558 billion won,"2,974 billion won","2,616 billion won",850 billion won,"8,576 billion won","6,945 billion won","2,660 billion won",847 billion won,"7,007 billion won"
Amount accepted,"2,602 billion won","1,700 billion won","1,000 billion won",508 billion won,"1,650 billion won","1,850 billion won","1,650 billion won","2,150 billion won",506 billion won,"1,650 billion won",...,500 billion won,350 billion won,"1,000 billion won",950 billion won,300 billion won,"3,318 billion won","2,549 billion won",904 billion won,437 billion won,"2,600 billion won"
Bid‐to‐Cover Ratio,287.0%,298.5%,339.9%,316.4%,293.2%,272.5%,303.1%,296.8%,321.8%,316.0%,...,282.2%,159.4%,297.4%,275.4%,283.3%,259.9%,277.8%,295.6%,211.8%,269.5%
